In [ ]:
!pip cache purge
!pip install imapclient openai langchain
!pip install python-dotenv

!pip install -U langchain-community
!pip install langchain-openai
!pip install langchain-chroma

!pip install faiss-cpu

### **Get emails "UNSEEN"**

In [ ]:
import imapclient
import email
from email.header import decode_header
import openai
import datetime
from bs4 import BeautifulSoup

EMAIL = "nu330006@gmail.com"
PASSWORD = "kund pbor krgm rzhx"

mail = imapclient.IMAPClient("imap.gmail.com", ssl=True)
mail.login(EMAIL, PASSWORD)

mail.select_folder("INBOX")

days = 30
while True:
    days_ago = (datetime.datetime.now() - datetime.timedelta(days=days)).strftime("%d-%b-%Y")
    messages = mail.search(["SINCE", days_ago, "UNSEEN"])

    if len(messages) > 0 or days >= 600:
        break
    days += 10

print(f"🔍 Đang kiểm tra email chưa đọc trong {days} ngày qua...")


if mail.has_capability('SORT'):
    messages = mail.sort(["REVERSE DATE"], messages)
else:
    fetch_data = mail.fetch(messages, ['INTERNALDATE'])
    messages_with_dates = [(msg_id, fetch_data[msg_id][b'INTERNALDATE']) for msg_id in messages]
    messages_with_dates.sort(key=lambda item: item[1], reverse=True)
    messages = [item[0] for item in messages_with_dates]

email_data = []


from email.header import decode_header
import chardet

def decode_email_content(content, encoding=None):
    if encoding:
        try:
            return content.decode(encoding)
        except UnicodeDecodeError:
            pass
    detected_encoding = chardet.detect(content)['encoding']
    if detected_encoding:
        return content.decode(detected_encoding)
    return content.decode('utf-8', errors='ignore')


for msg_id in messages:
    raw_msg = mail.fetch(msg_id, ["RFC822"])[msg_id][b"RFC822"]
    msg = email.message_from_bytes(raw_msg)

    from_email = msg["From"]
    if isinstance(from_email, tuple):
        from_email = " ".join(str(part) for part in from_email)

    subject, encoding = decode_header(msg["Subject"])[0]
    subject = subject.decode(encoding) if encoding else subject

    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if content_type == "text/plain" and "attachment" not in content_disposition:
                body = part.get_payload(decode=True)
                body = decode_email_content(body, part.get_content_charset())
                break
            elif content_type == "text/html" and "attachment" not in content_disposition:
                html_content = part.get_payload(decode=True)
                html_content = decode_email_content(html_content, part.get_content_charset())

                soup = BeautifulSoup(html_content, "lxml")

                for tag in soup(['style', 'script', 'head', 'title', 'meta', 'link',
                                'noscript', 'header', 'footer', 'table', 'tr', 'td',
                                'th', 'tbody', 'thead', 'form', 'input']):
                    tag.decompose()

                for element in soup.find_all(['p', 'div', 'br', 'li', 'h1', 'h2', 'h3']):
                    element.append('\n')

                body = soup.get_text(separator='\n', strip=True)

                import re
                body = re.sub(r'\n{3,}', '\n\n', body)
                body = re.sub(r'[\xa0\u200b]+', ' ', body)
    else:
        body = msg.get_payload(decode=True)
        body = decode_email_content(body, msg.get_content_charset())

    email_data.append({"From email": from_email, "subject": subject, "body": body})

for email in email_data:
    print(f"📩 **From:** {email['From email']}")
    print(f"📩 **Tiêu đề:** {email['subject']}")
    print(f"📜 **Nội dung:** {email['body']}...")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving API Key.env to API Key (1).env


### **Summarize body mail**

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv("/content/API Key.env")
google_api_key = os.getenv('GEMINI_API_KEY')


def generate_summary(body_mail):
    system_prompt = f"""You are a professional email summarization assistant. Summarize the email content into three sections::
    1. Main Purpose
    2. Key Information

    This mail: {body_mail}
    """

    genai.configure(api_key=google_api_key)
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(system_prompt)

    return response.text

In [ ]:
for email in email_data:
    summary = generate_summary(email["body"])
    print(f"📩 **From:** {email['From email']}")
    print(f"📩 **Tiêu đề:** {email['subject']}")
    print(f"✅ **Tóm tắt:** {summary}")
    print("\n")

    break

In [ ]:
path = "/content/drive/MyDrive/emails_output.txt"

with open(path, "w") as file:
    for email in email_data:
        summary = generate_summary(email["body"])
        mail = f"""
            **From:** {email['From email']}\n
            **Title:** {email['subject']}\n
            **Body:** {summary}\n\n"""
        file.write(mail)

In [ ]:
with open(path, "r") as file:
    content = file.read()
    print(content)

### **Split chunk to vector and store to vectordata**

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.schema import Document

loader = TextLoader("/content/drive/MyDrive/emails_output.txt")
documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

full_text = "\n\n".join(doc.page_content for doc in documents)
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

chunks = text_splitter.split_text(full_text)

In [ ]:
final_chunks = []
for chunk in chunks:
    header_start = chunk.find("📩 **From:**")
    metadata = {
        "source": "emails_output.txt",
        "email_header": chunk[header_start:].split("\n")[0].split(":", 1)[1].strip() if header_start != -1 else None
    }
    final_chunks.append(Document(page_content=chunk, metadata=metadata))

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(final_chunks, embedding=embeddings)

vectorstore.save_local("/content/drive/MyDrive/faiss_index")

loaded_vectorstore = FAISS.load_local("/content/drive/MyDrive/faiss_index", embeddings, allow_dangerous_deserialization=True) # Use allow_dangerous_deserialization here instead

total_vectors = loaded_vectorstore.index.ntotal
dimensions = loaded_vectorstore.index.d
print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the loaded vector store")

There are 7 vectors with 1,536 dimensions in the loaded vector store


### **Response body mail related to query**

In [ ]:
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

def query_vector_db(query):
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.5)

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    retriever = vectorstore.as_retriever()

    qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

    result = qa.invoke({"question": query})
    return result

In [ ]:
bodymail_relate = query_vector_db("List all emails about AI, and return email & body")

In [ ]:
bodymail_relate["answer"]

In [ ]:
import re

def extract_body_mail(text):
    email_entries = re.split(r"\n\n(?=\d+\. \*\*From:\*\*)", text.strip())

    dictt_email_body = {}

    for entry in email_entries:
        from_match = re.search(r"\*\*From:\*\* (.*)", entry)
        body_match = re.search(r"\*\*Body:\*\* (.*)", entry, re.DOTALL)

        if from_match and body_match:
            sender = from_match.group(1).strip()
            body = body_match.group(1).strip()

            print("From:", sender)
            print("Body:", body)
            print("------")

            dictt_email_body[sender] = body

    return dictt_email_body

extract_body_mail(bodymail_relate["answer"])

### **Create an Automatic Reply**

In [ ]:
from langchain_openai import ChatOpenAI

def generate_auto_reply(email_sender_name, email_body):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
    prompt = f"""
    You are a professional virtual assistant named **Huynh Minh Tam**.
    Please create a short and polite email response based on the following content:

    - Sender: {email_sender_name}
    - Email content: {email_body}

    Your response should:
    1. Address the sender by their name (if available).
    2. Be concise and professional.
    3. Sign off with your name: **Huynh Minh Tam**.

    Response:
    """
    response = llm.invoke(prompt)
    return response.content


def save_auto_replies(filename):
    with open(filename, "w", encoding="utf-8") as file:
        for sender_name, body_mail in extract_body_mail(bodymail_relate["answer"]).items():
            auto_reply = generate_auto_reply(sender_name, body_mail)
            file.write(f"To: {sender_name}\n")
            file.write(f"From: Huynh Minh Tam <nu330006@gmail.com>\n")
            file.write(f"Auto-reply: {auto_reply}\n")
            file.write("------\n")


save_auto_replies("/content/auto_replies.txt")

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(sender_email, receiver_email, subject, body, smtp_server, smtp_port, smtp_username, smtp_password):
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_username, smtp_password)
            server.send_message(msg)
        print(f"Email sent to {receiver_email}")
    except Exception as e:
        print(f"Failed to send email to {receiver_email}: {e}")


smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_username = "nu330006@gmail.com"
smtp_password = "kund pbor krgm rzhx"


def send_auto_replies_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        content = file.read()

    replies = content.split("------")

    for reply in replies:
        if not reply.strip():
            continue

        to_match = re.search(r"To: (.*)", reply)
        from_match = re.search(r"From: (.*)", reply)
        subject_match = re.search(r"Subject: (.*)", reply)
        body_match = re.search(r"Auto-reply: (.*)", reply, re.DOTALL)

        if to_match and from_match and body_match:
            receiver_email = to_match.group(1).strip()
            sender_email = from_match.group(1).strip()
            subject = subject_match.group(1).strip() if subject_match else "Auto-reply: Thank you for your email"
            body = body_match.group(1).strip()

            send_email(
                sender_email=sender_email,
                receiver_email=receiver_email,
                subject=subject,
                body=body,
                smtp_server=smtp_server,
                smtp_port=smtp_port,
                smtp_username=smtp_username,
                smtp_password=smtp_password
            )

send_auto_replies_from_file("/content/auto_replies.txt")

Email sent to Cohere Team <hello@cohere.com>
